# Exercises and Business Applications

Add in more tools - perhaps to simulate actually booking a flight. A student has done this and provided their example in the community contributions folder.

Next: take this and apply it to your business. Make a multi-modal AI assistant with tools that could carry out an activity for your work. A customer support assistant? New employee onboarding assistant? So many possibilities! Also, see the week2 end of week Exercise in the separate Notebook.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a HUGE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>

In [ ]:
# ------------------------------------------------------------------
# Southern Home – Dish-First Multimodal Cooking Assistant
# ------------------------------------------------------------------
#
# This notebook implements a multimodal AI chatbot for a local grocer
# called "Southern Home", focused on Southern-inspired home cooking.
#
# The assistant follows a dish-first interaction model:
# - It prompts users to choose or name a Southern-style dish
# - Suggests a concise list of nutritionally dense, simple ingredients
#   that are commonly available at a local grocery store
# - Recommends substitutes or larger retailers (e.g. Kroger, Walmart)
#   when ingredients are not available in-store
# - Provides cooking steps ONLY when explicitly requested by the user
#
# Multimodal capabilities:
# - Text generation via OpenAI GPT-4.1 Mini
# - Text-to-speech audio responses using OpenAI Mini TTS
# - Image generation of the suggested dish using DALL·E 3
#
# Data layer:
# - Uses a local SQLite database (supermarket.db)
# - Stores grocery items and prices
# - Database is automatically created and seeded on startup
#
# User Interface:
# - Built with Gradio Blocks
# - Supports free-text input and one-click dish suggestions
# - Displays chat responses, generated audio, and dish imagery
#
# LLMs & Models Used:
# - GPT-4.1 Mini (conversation, reasoning, dish & ingredient suggestions)
# - DALL·E 3 (dish image generation)
# - GPT-4o Mini TTS (audio output)
#
# Design goals:
# - Friendly Southern Belle tone
# - Short, clear, accurate responses
# - Emphasis on simple, wholesome, home-style cooking
# - Safe handling of missing data and user uncertainty
#
# ------------------------------------------------------------------

# ----------------------------------
# Imports
# ----------------------------------

import os
import json
import sqlite3
import base64
from io import BytesIO
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
from PIL import Image
import uuid

# ----------------------------------
# Initialization
# ----------------------------------

load_dotenv(override=True)

MODEL = "gpt-4.1-mini"
openai = OpenAI()

DB = "supermarket.db"

with sqlite3.connect(DB) as conn:
    cursor = conn.cursor()
    cursor.execute(
        "CREATE TABLE IF NOT EXISTS prices (item TEXT PRIMARY KEY, price REAL)"
    )
    conn.commit()

# ----------------------------------
# System Message (Dish-First Cooking Assistant)
# ----------------------------------

system_message = """
You are a Southern home cooking assistant for a local grocer called Southern Home.

Start by asking what Southern-inspired dish the user would like to cook today.
When a dish is mentioned:
- Suggest a short list of essential ingredients available in-store.
- Favor nutritionally dense, simple, home-style cooking.
- Do NOT provide cooking steps unless explicitly asked.

If an ingredient is unavailable in-store:
- Suggest reasonable substitutes the store carries, OR
- Recommend purchasing the item from a larger retailer such as Kroger or Walmart.

Keep answers friendly, warm, and concise with a Kentucky.
"""

# ----------------------------------
# Set Item Prices in Empty Database
# ----------------------------------

def set_item_price(item, price):
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute(
            "INSERT INTO prices (item, price) VALUES (?, ?) "
            "ON CONFLICT(item) DO UPDATE SET price = ?",
            (item.lower(), price, price)
        )
        conn.commit()

# ----------------------------------
# Database Helpers
# ----------------------------------

def get_item_price(item):
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute(
            "SELECT price FROM prices WHERE item = ?",
            (item.lower(),)
        )
        result = cursor.fetchone()
    return result[0] if result else None

def list_available_items(items):
    available = []
    unavailable = []

    for item in items:
        if get_item_price(item):
            available.append(item)
        else:
            unavailable.append(item)

    return available, unavailable

def extract_candidate_ingredients(text):
    """
    Very simple heuristic: match known grocery items in text.
    Avoids hallucination.
    """
    found = []
    for item in grocery_prices.keys():
        if item in text.lower():
            found.append(item)
    return found

def format_ingredient_availability(ingredients):
    available, unavailable = list_available_items(ingredients)

    lines = []

    if available:
        lines.append("🛒 **Available at Southern Home:**")
        for item in available:
            price = get_item_price(item)
            lines.append(f"- {item.title()} — ${price:.2f}")

    if unavailable:
        lines.append("\n❌ **Not currently in-store:**")
        for item in unavailable:
            lines.append(
                f"- {item.title()} (try Kroger or Walmart, or ask for substitutes)"
            )

    return "\n".join(lines)

# ----------------------------------
# Create Order Number
# ----------------------------------

def generate_order_id():
    return f"SH-{uuid.uuid4().hex[:8].upper()}"

# ----------------------------------
# Seed Data
# ----------------------------------

grocery_prices = {
    "apples": 3.49,
    "peaches": 3.99,
    "strawberries": 4.49,
    "cherry tomatoes": 3.29,
    "sweet corn": 2.99,
    "green beans": 3.19,
    "zucchini": 2.79,
    "yellow squash": 2.79,
    "potatoes": 3.49,
    "onions": 2.49,
    "eggs": 5.49,
    "milk": 4.29,
    "butter": 5.99,
    "cheddar cheese": 6.49,
    "yogurt": 4.99,
    "sourdough bread": 5.49,
    "honey": 7.99,
    "sorghum syrup": 6.99,
    "maple syrup": 8.99,
    "strawberry jam": 5.99,
    "ground beef": 7.99,
    "chicken thighs": 6.49,
    "lamb chops": 7.49,
    "smoked sausage": 6.99,
    "bacon": 7.99,
    "pinto beans": 2.49,
    "cornmeal": 3.29,
    "all-purpose flour": 3.49,
    "rice": 4.29,
    "pasta": 3.19,
    "fresh basil": 2.99,
    "fresh thyme": 2.79,
    "fresh rosemary": 2.79,
    "garlic": 2.49,
    "mushrooms": 3.99
}

for item, price in grocery_prices.items():
    set_item_price(item, price)

# ----------------------------------
# Image Generation (Dish-Centric)
# ----------------------------------

def artist(dish):
    image_response = openai.images.generate(
        model="dall-e-3",
        prompt=(
            f"A comforting Southern-style {dish}, home-cooked, warm lighting, "
            "rustic plating, nutritionally wholesome, Studio Ghibli-style illustration"
        ),
        size="1024x1024",
        n=1,
        response_format="b64_json"
    )

    image_data = base64.b64decode(image_response.data[0].b64_json)
    return Image.open(BytesIO(image_data))

# ----------------------------------
# Text-to-Speech
# ----------------------------------

def talker(message):
    response = openai.audio.speech.create(
        model="gpt-4o-mini-tts",
        voice="coral",
        input=message
    )
    return response.content

# ----------------------------------
# Core Chat Logic (Dish-First)
# ----------------------------------

def chat(history):
    history = [{"role": h["role"], "content": h["content"]} for h in history]

    messages = [{"role": "system", "content": system_message}] + history

    response = openai.chat.completions.create(
        model=MODEL,
        messages=messages
    )

    reply = response.choices[0].message.content
    history.append({"role": "assistant", "content": reply})

    # 🔊 Audio should ALWAYS play
    voice = talker(reply)

    # 🧾 Ingredient grounding & ordering (only when ingredients appear)
    ingredients = extract_candidate_ingredients(reply)

    if ingredients:
        order_id = generate_order_id()
        availability = format_ingredient_availability(ingredients)

        order_info = (
            f"\n\n{availability}\n\n"
            f"🧾 **Order Number:** {order_id}\n"
            "Would you like to order any of these from Southern Home today?"
        )

        reply = reply + order_info
        history[-1]["content"] = reply

    # 🖼️ Image generation ONLY when USER mentions a dish
    image = None
    last_user_message = history[-2]["content"].lower() if len(history) >= 2 else ""

    known_dishes = [
        "shrimp and grits",
        "chicken and vegetable skillet",
        "red beans and rice"
    ]

    for dish in known_dishes:
        if dish in last_user_message:
            image = artist(dish)
            break

    return history, voice, image

def put_message_in_chatbot(message, history):
    return "", history + [{"role": "user", "content": message}]

def dish_button_click(dish, history):
    return "", history + [{"role": "user", "content": dish}]


# ----------------------------------
# Gradio UI
# ----------------------------------

with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
        image_output = gr.Image(height=500, interactive=False)

    with gr.Row():
        audio_output = gr.Audio(autoplay=True)

    with gr.Row():
        message = gr.Textbox(label="What Southern dish are you fixin’ to cook today?")

    # Suggested dishes
    with gr.Row():
        shrimp_grits = gr.Button("🍤 Shrimp and Grits")
        chicken_skillet = gr.Button("🍗 Chicken & Vegetable Skillet")
        red_beans = gr.Button("🥘 Red Beans and Rice")

    # Text input submission
    message.submit(
        put_message_in_chatbot,
        inputs=[message, chatbot],
        outputs=[message, chatbot]
    ).then(
        chat,
        inputs=chatbot,
        outputs=[chatbot, audio_output, image_output]
    )

    # Button interactions
    shrimp_grits.click(
        dish_button_click,
        inputs=[shrimp_grits, chatbot],
        outputs=[message, chatbot]
    ).then(
        chat,
        inputs=chatbot,
        outputs=[chatbot, audio_output, image_output]
    )

    chicken_skillet.click(
        dish_button_click,
        inputs=[chicken_skillet, chatbot],
        outputs=[message, chatbot]
    ).then(
        chat,
        inputs=chatbot,
        outputs=[chatbot, audio_output, image_output]
    )

    red_beans.click(
        dish_button_click,
        inputs=[red_beans, chatbot],
        outputs=[message, chatbot]
    ).then(
        chat,
        inputs=chatbot,
        outputs=[chatbot, audio_output, image_output]
    )

ui.launch(inbrowser=True)

